# Imports

In [18]:
import requests
from flask import jsonify
import json
from datetime import date, time, datetime
from seleniumrequests import Firefox, PhantomJS
from bs4 import BeautifulSoup
from selenium import webdriver


# Testing POST /merge_pdfs endpoint

In [2]:
body_all = {"1": {"bp_bank_id": "0170",
             "last_updated": "2021-04-06",
             "url": "https://www.abanca.pt"
             },
             "2": {"bp_bank_id": "0269",
             "last_updated": "2021-04-06",
             "url": "https://www.bankinter.pt"
             },
             "3": {"bp_bank_id": "0079",
             "last_updated": "2021-04-06",
             "url": "https://www.bancobic.ao"
             },
             "4": {"bp_bank_id": "0008",
             "last_updated": "2021-04-06",
             "url": "https://www.bancobai.ao"
             },
             "5": {"bp_bank_id": "0193",
             "last_updated": "2021-04-06",
             "url": "https://www.bancoctt.pt/"
             }
            }

In [3]:
# requires this request body
body = {"1": {"bp_bank_id": "0170",
  "last_updated": "2021-04-06",
  "url": "https://www.abanca.pt"
  }
}

In [4]:
# setting urls for localhost and heroku
url = "http://127.0.0.1:5000/merge_pdfs"
url_h = 'https://bank-price-api.herokuapp.com/merge_pdfs'

In [5]:
response = requests.post(url, data = None, json = body).json()

In [6]:
ident = response.get('ident')

In [7]:
ident

999

## Testing GET /retrievepdfs

Since the ```/merge_pdfs``` job can take some time and we want to avoid heroku timeout issues the job is done in the background. To pick up the requested file, the rails app can go to this endpoint after some waiting time. 
There will be only one required argument ```ident``` to be passed to the GET request which was given before by the response of ```/merge_pdfs```.

In [8]:
# setting urls for localhost and heroku
url = "http://127.0.0.1:5000/retrievepdfs"
url_h = 'https://bank-price-api.herokuapp.com/retrievepdfs'

In [9]:
params = {'ident':ident}

In [10]:
# call this after logs say: 'updated bank.json ready for pickup with ident: <ident>'
response = requests.get(url, params=params).json()

In [11]:
response

{'message': "sourcing job not finished or initialized or ident: 999 is not available. first call /merge_pdfs and wait for backgroundjob to finish. Error msg: [Errno 2] No such file or directory: 'bank_benchmark_api/data/banks_999.json'",
 'status': 'error'}

In [12]:
# saving infos for the other endpoints
bp_pdf_url = response.get('1').get('bp_pdf_url')
bp_bank_id = response.get('1').get('bp_bank_id')

AttributeError: 'NoneType' object has no attribute 'get'

# Testing POST /get_stats

In [ ]:
# requires this request body
body = {
    "1": {
        "url": "www.abanca.pt",
        "bp_pdf_url": bp_pdf_url,
        "bp_bank_id": bp_bank_id,
        "cloud_merged_url": "https://storage.googleapis.com/bank_price_pdfs/1_all_products_210412130040.pdf",
        "products": {"demand_deposit":
                {
                    "statement": [
                    "Emissão de extrato",
                    "Extrato Integrado",
                    "Extrato Mensal",
                    "Extrato integrado",
                    "Extrato avulso"
                ],
                "documents_copy": [
                    "Fotocópias de segundas vias de talões de depósito",
                    "Emissão 2ªs Vias de Avisos e Outros Documentos",
                    "Extracto avulso",
                    "Segundas vias (pedido na agência)"
                ],
                "acc_manteinance": [
                    "Manutenção de conta",
                    "Comissão de manutenção de conta",
                    "Comissão de Manutenção de Conta",
                    "Manutenção de Conta Pacote",
                    "Manutenção de Conta Base",
                    "Manutenção de Conta Serviços Mínimos Bancários"
                ],
                "withdraw": [
                    "Levantamento de numerário",
                    "Levantamento de numerário ao balcão",
                    "Comissão de Levantamento",
                    "Levantamento de Numerário ao Balcão",
                    "Levantamento de Numerário ao Balcão"
                ],
                "online_service": [
                    "Adesão ao serviço de banca à distância",
                    "Adesão ao serviço online"
                ],
                "cash_deposit": [
                    "Depósito de moedas metálicas",
                    "Depósito de moedas",
                    " Depósito em moeda metálica (>= 100 moedas)",
                    "Depósito de moedas ao balcão",
                    "Depósito de dinheiro ao balcão",
                    "Depósito em moeda metálica (>= 100 moedas)"
                ],
                "change_holder": [
                    "Alteração de titulares",
                    "Alteração de titularidade",
                    "Comissão de Alteração de Titularidade",
                    "Alteração de titularidade / intervenientes",
                    "Alteração de titularidade (titular/ representante)"
                ],
                "bank_overdraft": [
                    "Comissões por descoberto bancário",
                    "Descoberto bancário",
                    "Comissões por Descoberto Bancário"
                ],
                "movement_consultation": [
                    "Consulta de Movimentos de conta DO com",
                    "Consulta de movimentos ao balcão"
                ],
                "balance_inquiry": [
                    "Pedido de saldo ao balcão",
                    "Consulta de Saldo de conta DO com comprovativo"
                ],
                "portuguese": "Contas de Depósito"
            }
        }
    }
}

In [ ]:
# setting urls for localhost and heroku
url = "http://127.0.0.1:5000/get_stats"
url_h = 'https://bank-price-api.herokuapp.com/get_stats'

In [ ]:
response = requests.post(url, data = None, json = body).json()

In [ ]:
ident_s = response.get('ident')

In [ ]:
ident_s

## Testing GET /retrievestats

Since the ```/get_stats``` job can take some time and we want to avoid heroku timeout issues the job is done in the background. To pick up the requested file, the rails app can go to this endpoint after some waiting time. 
There will be only one required argument ```ident``` to be passed to the GET request which was given before by the response of ```/get_stats```.

In [ ]:
# setting urls for localhost and heroku
url = "http://127.0.0.1:5000/retrievestats"
url_h = 'https://bank-price-api.herokuapp.com/retrievestats'

In [ ]:
params = {'ident':ident_s}

In [ ]:
# call this after logs say: 'bank price json with <ident> loaded, supplied and removed from server'
response = requests.get(url, params=params).json()

In [ ]:
response

# Special cases selenium

In [2]:
proxies = {
 "http": "http://185.51.92.108:51327",
 "https": "http://185.51.92.108:51327",
}
headers = {'Accept-Language': 'pt-PT'}

In [33]:
## Bancobai doesnt allow automatized parsing: 
url = 'https://www.bancobaieuropa.pt'
# response = requests.get(url, headers=headers, verify=False)

In [39]:
PHANTOMJS_PATH = '/opt/homebrew/bin/geckodriver'
driver = webdriver.Firefox()
driver.get(url)
driver.implicitly_wait(8)
body = driver.page_source

In [36]:
# let's parse our html

soup=BeautifulSoup(driver.page_source)

In [37]:
soup

<html class="cookies no-ie8compat serviceworker svg websockets canvas canvastext no-userdata video multiplebgs rgba placeholder inlinesvg ambientlight no-inputsearchevent csscalc cssgradients opacity supports svgasimg no-touchevents checked displaytable display-table fontface cssinvalid lastchild no-nthchild mediaqueries textshadow bgpositionxy backgroundsize bgsizecover borderimage borderradius boxshadow boxsizing flexbox flexboxlegacy no-flexboxtweener flexwrap csstransforms csstransforms3d preserve3d csstransitions objectfit object-fit csshyphens softhyphens no-softhyphensfind" dir="ltr" lang="pt-pt" prefix="content: http://purl.org/rss/1.0/modules/content/  dc: http://purl.org/dc/terms/  foaf: http://xmlns.com/foaf/0.1/  og: http://ogp.me/ns#  rdfs: http://www.w3.org/2000/01/rdf-schema#  schema: http://schema.org/  sioc: http://rdfs.org/sioc/ns#  sioct: http://rdfs.org/sioc/types#  skos: http://www.w3.org/2004/02/skos/core#  xsd: http://www.w3.org/2001/XMLSchema# "><head>
<meta cha

In [38]:
search = ['preçário', 'pricelist', 'precario']
for link in soup.find_all('a', href=True):
    url_prices = str(link.get('href').lower().strip())
    lower = str(link.string).lower().strip()
    title = str(link.get('title')).strip().lower()
    searchstring = ' '.join([url_prices, lower, title])
    if any([x in searchstring for x in search]):
        print(f'found terms of {search} in string {searchstring}')

found terms of ['preçário', 'pricelist', 'precario'] in string /pt-pt/informacoes/precario preçário none
found terms of ['preçário', 'pricelist', 'precario'] in string /informacoes/precario none none
found terms of ['preçário', 'pricelist', 'precario'] in string /pt-pt/informacoes/precario preçário none


In [ ]:
soup